In [ ]:
import numpy as np 
import qutip as qt
from projections_CPTP import initialization, exactSDP, cholesky_based_approximation, dykstra_cholesky_based_approximation, dykstra_identity, HIP, two_stage_solution

## Data


In [ ]:
import pickle

# Load exact data SDP 
with open('data/dataExactChoi_p1e3.pkl', 'rb') as pickle_file:
    data_sdp = pickle.load(pickle_file)

In [ ]:
def generate_samples(data_sdp, qubits, iters, tol, reps, data):
    """
    Generate samples for quantum operations based on provided data and parameters.

    Args:
    - data_sdp (dict): Dictionary containing SDP data.
    - qubits (int): Number of qubits for the quantum operation.
    - iters (int): Number of iterations for iterative algorithms.
    - tol (float): Tolerance level for convergence criteria.
    - reps (int): Number of repetitions for sampling.
    - data (dict): Dictionary to store generated data.

    Returns:
    - data (dict): Dictionary containing generated data.

    This function iterates over the specified number of repetitions and performs quantum operations
    based on the provided data and parameters. It calculates distances and times for each operation
    compared to exact values and stores them in the data dictionary.
    """

    for k in range(reps):
        # Extract input and exact data for the current repetition
        rhoInput = np.array(data_sdp[f"data_{qubits}qubits"]['rhoInput'][k])
        rhoExact = np.array(data_sdp[f"data_{qubits}qubits"]['rhoOutSDP'][k])
        
        # Perform quantum operations and measure time taken
        rhoCBA, timeCBA = cholesky_based_approximation(rhoInput, pos=False)
        rhoTSS, timeTSS = two_stage_solution(rhoInput, pos=False)
        rhoDykstraCBA, timeDykstraCBA, itDykstraCBA = dykstra_cholesky_based_approximation(rhoInput, iters, tol)
        rhoHIP, timeHIP = HIP(rhoInput, Id=True)
        rhoDykstraId, timeDykstraId, itDykstraId = dykstra_identity(rhoInput, iters, tol)
        
        # Calculate distances between results and exact values and store in data dictionary
        data['distanceCBA'][k] = np.linalg.norm(rhoCBA - rhoExact, "fro")
        data['timeCBA'][k] = timeCBA
        data['distanceTSS'][k] = np.linalg.norm(rhoTSS - rhoExact, "fro")
        data['timeTSS'][k] = timeTSS
        data['distanceDykstraCBA'][k] = np.linalg.norm(rhoDykstraCBA - rhoExact, "fro")
        data['timeDykstraCBA'][k] = timeDykstraCBA
        data['itersDykstraCBA'][k] = itDykstraCBA
        data['distanceHIP'][k] = np.linalg.norm(rhoHIP - rhoExact, "fro")
        data['timeHIP'][k] = timeHIP
        data['distanceDykstraId'][k] = np.linalg.norm(rhoDykstraId - rhoExact, "fro")
        data['timeDykstraId'][k] = timeDykstraId
        data['itersDykstraId'][k] = itDykstraId

    return data


In [ ]:
def generate_samples_qubits(data_sdp, qubits_list, iters, tol, reps):
    """
    Generate samples for quantum operations with varying qubit numbers.

    Args:
    - data_sdp (dict): Dictionary containing SDP data.
    - qubits_list (list): List of qubit numbers for quantum operations.
    - iters (int): Number of iterations for iterative algorithms.
    - tol (float): Tolerance level for convergence criteria.
    - reps (int): Number of repetitions for sampling.

    Returns:
    - data_qubits (dict): Dictionary containing generated data for each qubit configuration.

    This function iterates over a list of qubit numbers, generates data for each qubit configuration,
    and stores the results in a dictionary. It prints the progress as it runs for each qubit configuration.
    """

    data_qubits = {}
    for qubits in qubits_list:
        print(f"Running for {qubits} qubits... ")
        # Initialize data dictionary for the current qubit configuration
        data = {'distanceCBA': np.zeros((reps)), 'timeCBA': np.zeros((reps)),
                'distanceTSS': np.zeros((reps)), 'timeTSS': np.zeros((reps)),
                'distanceDykstraCBA': np.zeros((reps)), 'timeDykstraCBA': np.zeros((reps)),
                'itersDykstraCBA': np.zeros((reps)), 'distanceHIP': np.zeros((reps)),
                'timeHIP': np.zeros((reps)), 'distanceDykstraId': np.zeros((reps)),
                'timeDykstraId': np.zeros((reps)), 'itersDykstraId': np.zeros((reps))}
        # Generate samples for the current qubit configuration and update data dictionary
        data = generate_samples(data_sdp, qubits, iters, tol, reps, data)
        data_qubits[f'data_{qubits}qubits'] = data

    return data_qubits


In [ ]:
iters = 100
tol = 1e-7
reps = 100
qubits_list = [2, 4, 6, 8]
data_qubits = generate_samples_qubits(data_sdp, qubits_list, iters, tol, reps)

In [ ]:
import pickle

# Save dictionary to a file
with open('data/dataChoiCholesky_p1e3.pkl', 'wb') as pickle_file:
    pickle.dump(data_qubits, pickle_file)
